In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Options
import numpy as np
import sys
import gamspy as gp
import math
import matplotlib.pyplot as plt
import pickle
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle

In [430]:
import numpy as np
import pandas as pd

# Step 1: Define parameters
num_stocks = 50         # Total number of stocks
num_increasing = 45      # Number of stocks with increasing prices
num_decreasing = 5      # Number of stocks with decreasing prices
num_months = 6          # Number of months (time periods)
initial_price = 100      # Initial price for all stocks

# Ensure the total number of stocks matches the sum of increasing and decreasing stocks
assert num_increasing + num_decreasing == num_stocks, "The sum of increasing and decreasing stocks must equal the total number of stocks."

# Step 2: Generate fake price data
price_array = np.zeros((num_stocks, num_months))

# Generate increasing prices for the specified number of stocks
for i in range(num_increasing):
    # Simulate an increasing trend by adding a random increment each month
    price_array[i] = initial_price + np.cumsum(np.random.uniform(1, 5, num_months))

# Generate decreasing prices for the specified number of stocks
for i in range(num_increasing, num_increasing + num_decreasing):
    # Simulate a decreasing trend by subtracting a random decrement each month
    price_array[i] = initial_price - np.cumsum(np.random.uniform(1, 5, num_months))

# Ensure no negative prices (optional, depending on the simulation)
price_array = np.maximum(price_array, 0)

# Step 3: Convert the price array into a DataFrame for better visualization
tickers = [f"Stock_{i+1}" for i in range(num_stocks)]
months = [f"Month_{i+1}" for i in range(num_months)]

price_df = pd.DataFrame(price_array, index=tickers, columns=months)

# Step 4: Print the resulting array and DataFrame
print("Price array shape:", price_array.shape)  # Should be (num_stocks, num_months)
print("Fake Price Array:")
print(price_array)

print("\nFake Price DataFrame:")
print(price_df)

# Optional: Save the fake price data to a CSV file
price_df.to_csv('fake_stock_prices.csv')

Price array shape: (50, 6)
Fake Price Array:
[[101.75522166 105.73898684 109.67418267 112.87911112 115.69505878
  116.87130523]
 [102.54926606 104.78562822 108.57273193 112.47158631 116.85787584
  120.33965975]
 [101.65104879 104.56119133 107.37115178 112.23972339 115.89723665
  117.06070586]
 [103.40231688 108.28484157 110.01850826 113.94684003 115.04557593
  116.35991527]
 [102.54325809 104.60571307 107.76871204 108.77598591 113.35173046
  115.02425062]
 [103.16446515 105.30877193 107.27815247 109.42855158 113.85969435
  118.26962185]
 [101.10265908 104.23457437 109.14627932 111.09854282 113.8645608
  118.49128505]
 [104.31257559 106.81648057 110.94685001 115.16452354 117.01785577
  121.71359932]
 [102.84625321 107.37841256 111.23297001 113.81237834 116.0265314
  119.94419162]
 [104.71352256 109.23394829 113.55217849 118.28797882 123.1171728
  127.30687392]
 [103.19161286 106.3053431  109.73201953 112.14500657 114.38498097
  117.91464756]
 [102.20476819 106.92456278 108.15468446 110.

In [431]:
# Calculate the percentage change for each stock
initial_prices = price_array[:, 0]  # Prices at Month_1 (initial prices)
final_prices = price_array[:, -1]   # Prices at Month_13 (final prices)

# Calculate returns for each stock
returns = ((final_prices - initial_prices) / initial_prices) * 100

# Calculate the average return
average_return = np.mean(returns)

print("Returns for each stock (%):")
print(returns)

print("\nAverage return across all stocks (%):")
print(average_return)


Returns for each stock (%):
[ 14.85533944  17.34814336  15.15936851  12.53124571  12.17144137
  14.64182137  17.19897986  16.68161642  16.62475576  21.57634545
  14.26766603  16.38828605  14.24155268  15.75280593  14.09194424
  15.26299258  17.45867975  14.01163045  14.63224861  11.55554528
  16.96125935  14.58852721   7.01872971  12.04451856  12.96102324
  14.04734478  14.76841293  12.48967998  16.07383359  10.23875981
  17.07354096  13.87031787  11.63861817  16.83867329  11.75883734
  12.5089086   15.81482513  13.07324546  17.05500425  13.70060763
  13.58036627  13.99840888  19.00302121  14.91838026  14.63255054
 -16.00882536 -16.72250708 -17.17196282 -16.5569922  -10.69902719]

Average return across all stocks (%):
11.59900978430246


In [481]:
# Portfolio Configuration Parameters
max_exposure = 0.10  # Maximum exposure per stock
buy_exposure = 0.10  # Maximum buying limit as a fraction of NAV
max_return = 1.80  # Cap on maximum return per stock
PT = 0.90 # Portfolio turnover rate
cash_min = 0.01  # Minimum cash position as a fraction of NAV
bfee = 1.01  # Buy fee multiplier
sfee = 0.99  # Sell fee multiplier
min_buy = 0.001  # Minimum purchase amount
expected_return = 1.09  # Target return for the portfolio
init_cash = 1000000  # Initial cash balance
eps = 0.0000001
M = 100000000
buy_minimum = 1000

# Define the model container
m = Container()

# Sets
time_index = num_months  # Total number of time periods
stocks = Set(container=m, name="stocks", records=tickers)  # Set of stocks
time = Set(container=m, name="time", records=np.arange(0, time_index, 1))  # Time periods
subtime = Set(container=m, domain=time, records=np.arange(1, time_index, 1))  # Excludes first period
subtime2 = Set(container=m, domain=time, records=np.arange(2, time_index, 1))  # Excludes first two periods

# Data: Stock prices indexed by stocks and time
prices = Parameter(
    container=m,
    name="prices",
    domain=[stocks, time],
    records=price_array,
)

# Variables
C = Variable(container=m, name="C", domain=time, type="Positive")  # Cash balance
V = Variable(container=m, name="V", domain=time, type="Positive")  # Net Asset Value
x = Variable(container=m, name="x", domain=[stocks, time], type="Positive")  # Holdings
x.fx[stocks, '0'] = 0  # No initial holdings

b = Variable(container=m, name="b", domain=[stocks, time], type="Positive")  # Stocks bought
bp = Variable(container=m, name="bp", domain=[stocks, time], type="SemiCont")  # Stocks bought value
sp = Variable(container=m, name="sp", domain=[stocks, time], type="Positive")  # Stocks sold value
b.fx[stocks, '0'] = 0  # No initial buying
bp.lo[stocks, time] = 20000
bp.up[stocks,time]=100000

s = Variable(container=m, name="s", domain=[stocks, time], type="Positive")  # Stocks sold
Z = Variable(container=m, name="Z", type="free")  # Objective variable
Sr = Variable(container=m, name="Sr", domain=[stocks], type="Positive")  # Total sales
Br = Variable(container=m, name="Br", domain=[stocks], type="Positive")  # Total purchases

gamma_s = Variable(container=m, name="gs", domain=[stocks, time], type="Binary")  # Total purchases
gamma_b = Variable(container=m, name="gb", domain=[stocks, time], type="Binary")  # Total purchases

profit = Variable(container=m, name="profit", domain=[stocks], type="Free")  # Profit per stock
omega = Variable(container=m, name="omega", domain=[stocks], type="Binary")  # Binary variable if stock i is a winner

## Binary variable if stock i was a trade
omega_trades = Variable(container=m, name="omega_stock", domain=[stocks], type="Binary")


# Equations
buy_or_sell = Equation(
    m,
    name="buy_or_sell",
    domain=[stocks, time],
    description="Determines whether the stock is sold or bought at time t"
)
buy_or_sell[stocks, time] = (1 >= gamma_s[stocks, time] + gamma_b[stocks, time])


## Determination of omega_trades for stock i
omega_trades_determination = Equation(
    m,
    name="omega_trades_determination",
    domain=[stocks],
    description="Determines if Br[stocks] > 1 and sets omega_trades to 1 in that case"
)
omega_trades_determination[stocks] = (
    Sum(time, bp[stocks,time]) <= M * omega_trades[stocks]
)


## Determination of omega_trades for stock i
omega_trades_determination2 = Equation(
    m,
    name="omega_trades_determination2",
    domain=[stocks],
    description="Determines if Br[stocks] > 1 and sets omega_trades to 1 in that case"
)
omega_trades_determination2[stocks] = (
    Sum(time, bp[stocks,time]) >= 0.001 * (omega_trades[stocks])
)





buy_limit = Equation(
    m,
    name="buy_limit",
    domain=[stocks, time],
    description="Binary variable that switches on if stock i is bought at time t"
)
buy_limit[stocks, time] = b[stocks, time] <= M * gamma_b[stocks,time]


sell_limit = Equation(
    m,
    name="sell_limit",
    domain=[stocks, time],
    description="Binary variable that switches on if stock i is sold at time t"
)
sell_limit[stocks, time] = s[stocks, time] <= M * gamma_s[stocks,time]





stocks_bought_value = Equation(
    m,
    name="stocks_bought_value",
    domain=[stocks, time],
    description="Determines the value of the stocks bought at time t"
)
stocks_bought_value[stocks, time] = (bp[stocks, time] == b[stocks, time] * prices[stocks, time])

stocks_sold_value = Equation(
    m,
    name="stocks_sold_value",
    domain=[stocks, time],
    description="Determines the value of the stocks sold at time t"
)
stocks_sold_value[stocks, time] = (sp[stocks, time] == s[stocks, time] * prices[stocks, time])

winner_determination = Equation(
    m,
    name="winner_determination",
    domain=[stocks],
    description="Determines if investing in stock i was a winner through the lifetime of the fund"
)
winner_determination[stocks] = (
    (Sr[stocks] + x[stocks, str(time_index - 1)] * prices[stocks, str(time_index - 1)]) >= 
    Br[stocks] + eps * omega[stocks] - M * (1 - omega[stocks])
)

loser_determination = Equation(
    m,
    name="loser_determination",
    domain=[stocks],
    description="Determines if investing in stock i was a loser through the lifetime of the fund"
)
loser_determination[stocks] = (
    (Sr[stocks] + x[stocks, str(time_index - 1)] * prices[stocks, str(time_index - 1)]) - Br[stocks] <= 
    - eps * (1 - omega[stocks]) + M * omega[stocks]
)



total_sales = Equation(
    m,
    name="total_sales",
    domain=[stocks],
    description="Total revenue from selling stock",
)
total_sales[stocks] = (Sr[stocks] == Sum(time, prices[stocks, time] * s[stocks, time]))

total_purchases = Equation(
    m,
    name="total_purchases",
    domain=[stocks],
    description="Total cost from buying stock",
)
total_purchases[stocks] = (Br[stocks] == Sum(time, prices[stocks, time] * b[stocks, time]))

pnl = Equation(
    m,
    name="pnl",
    domain=[stocks],
    description="Calculate profit for each stock",
)
pnl[stocks] = (
    profit[stocks] == 
    (Sr[stocks] + x[stocks, str(time_index - 1)] * prices[stocks, str(time_index - 1)]) - Br[stocks]
)

portfolio_turnover = Equation(
    m,
    name="portfolio_turnover",
    description="Portfolio turnover calculation",
)
portfolio_turnover[...] = (
    PT * (V["1"] + V[str(time_index - 1)]) == 
    Sum(stocks, Sum(subtime2, b[stocks, subtime2] * prices[stocks, subtime2])) * 2
)

max_returns = Equation(
    m,
    name="max_returns",
    domain=[stocks],
    description="Cap on maximum returns per stock",
)
max_returns[stocks] = (
    (Sr[stocks] + x[stocks, str(time_index - 1)] * prices[stocks, str(time_index - 1)]) <= 
    Br[stocks] * max_return
)

holdings_balance = Equation(
    m,
    name="holdings_balance",
    domain=[stocks, time],
    description="Holdings balance over time",
)
holdings_balance[stocks, subtime] = (
    x[stocks, subtime] == x[stocks, subtime - 1] + b[stocks, subtime] - s[stocks, subtime]
)

holdings_balance_initial = Equation(
    m,
    name="holdings_balance_initial",
    domain=[stocks],
    description="Initial stock holdings",
)
holdings_balance_initial[stocks] = (x[stocks, "0"] == 0)

cash_balance = Equation(
    m,
    name="cash_balance",
    domain=[time],
    description="Cash balance at each time period",
)
cash_balance[time] = (
    C[time] == C[time - 1] - 
    Sum(stocks, bfee * bp[stocks, time]) + 
    Sum(stocks, sfee * sp[stocks, time])
)

cash_balance_initial = Equation(
    m,
    name="cash_balance_initial",
    description="Initial cash balance",
)
cash_balance_initial[...] = (C["0"] == init_cash)

nav = Equation(
    m,
    name="nav",
    domain=[time],
    description="Net Asset Value calculation",
)
nav[time] = (V[time] == C[time] + Sum(stocks, prices[stocks, time] * x[stocks, time]))

risk_constraint = Equation(
    m,
    name="risk_constraint",
    domain=[stocks, time],
    description="Limit position exposure",
)
risk_constraint[stocks, time] = (max_exposure * V[time] >= prices[stocks, time] * x[stocks, time])

buy_risk_constraint = Equation(
    m,
    name="buy_risk_constraint",
    domain=[stocks, time],
    description="Limit buying exposure",
)
buy_risk_constraint[stocks, time] = (buy_exposure * V[time] >= prices[stocks, time] * b[stocks, time])

Z_plus = Variable(container=m, name="Z_plus", type="Positive")
Z_minus = Variable(container=m, name="Z_minus", type="Positive")

constraint_deviation = Equation(
    container=m,
    name="constraint_deviation",
)
constraint_deviation[...] =  (Sum(stocks, omega[stocks])) == Z_plus

constraint_deviation2 = Equation(
    container=m,
    name="constraint_deviation2",
)
constraint_deviation2[...] =  1 == Z_minus*(Sum(stocks, omega_trades[stocks]))


#
portfolio_returns = Equation(
    container=m,
    name="portfolio_returns",
)
portfolio_returns[...] = (V[str(time_index - 1)] - expected_return * init_cash == 0)


obj_function = Equation(
    container=m,
    name="obj_function",
)
obj_function[...] = (Z) == Z_plus*Z_minus #- Z_plus

# Model definition
b1 = Model(
    container=m,
    name="b1",
    equations=m.getEquations(),
    problem="MINLP",
    sense=Sense.MIN,
    objective=Z,
)

# Solve the model
gdx_path = m.gdxOutputPath()
b1.solve(
    output=sys.stdout,
    solver='XPRESS',
    options=Options(report_solution=1),
    solver_options={
        "reslim": "30",
        # "SolnPoolReplace": 2,
        # "SolnPoolIntensity": 4,
        # "SolnPoolPop": 2,
        # "PopulateLim": 50,
        # "solnpoolmerge": "mysol.gdx",
    }
)


--- Job _2f6e5411-a380-4ee5-8bbb-1aa5447db666.gms Start 02/08/25 21:08:57 48.6.1 67fbb04b DAX-DAC arm 64bit/macOS
--- Applying:
    /Users/sarangbalan/miniconda3/envs/py310/lib/python3.10/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    MINLP XPRESS
    Input /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpr8bijwl7/_2f6e5411-a380-4ee5-8bbb-1aa5447db666.gms
    Output /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpr8bijwl7/_2f6e5411-a380-4ee5-8bbb-1aa5447db666.lst
    ScrDir /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpr8bijwl7/tmpitt7ygm0/
    SysDir /Users/sarangbalan/miniconda3/envs/py310/lib/python3.10/site-packages/gamspy_base/
    LogOption 3
    Trace /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpr8bijwl7/_2f6e5411-a380-4ee5-8bbb-1aa5447db666.txt
    License "/Users/sarangbalan/Library/Application Support/GAMSPy/gamspy_license.txt"
    OptFile 1
    OptDir /var/folders/26/q1w3l1l13l353qb4dk_p70300000gn/T/tmpr8bijwl7/
    LimRow 0
    Li

[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Resource,Integer,0.305555555555556,2818,2365,MINLP,XPRESS,30.051


In [405]:
import gamspy.utils as utils
print(utils.getAvailableSolvers())

['BARON', 'CBC', 'CONOPT', 'CONOPT3', 'CONOPT4', 'CONVERT', 'COPT', 'CPLEX', 'DICOPT', 'EXAMINER', 'EXAMINER2', 'GUROBI', 'HIGHS', 'IPOPT', 'IPOPTH', 'KESTREL', 'KNITRO', 'MILES', 'MINOS', 'MOSEK', 'MPSGE', 'NLPEC', 'PATH', 'PATHNLP', 'RESHOP', 'SBB', 'SCIP', 'SHOT', 'SNOPT', 'SOPLEX', 'XPRESS']


In [478]:
omega.records.level.sum()

np.float64(17.0)

In [479]:
omega_trades.records.level.sum()

np.float64(20.0)

In [470]:
df = profit.records

# Filter out rows where 'level' is 0
non_zero_levels = df[df['level'] > 0]

# Find the lowest non-zero value in the 'level' column
lowest_non_zero = non_zero_levels['level'].count()

print("Lowest non-zero value in 'level':", lowest_non_zero)

Lowest non-zero value in 'level': 6


In [480]:
profit.records

,stocks,level,marginal,lower,upper,scale
0,Stock_1,8.857429e+04,NaN,-inf,inf,1.0
1,Stock_2,7.708398e+04,NaN,-inf,inf,1.0
2,Stock_3,0.000000e+00,NaN,-inf,inf,1.0
3,Stock_4,0.000000e+00,NaN,-inf,inf,1.0
4,Stock_5,0.000000e+00,NaN,-inf,inf,1.0
5,Stock_6,1.588939e+05,NaN,-inf,inf,1.0
6,Stock_7,0.000000e+00,NaN,-inf,inf,1.0
7,Stock_8,9.520792e+04,NaN,-inf,inf,1.0
8,Stock_9,8.044161e+04,NaN,-inf,inf,1.0
9,Stock_10,0.000000e+00,NaN,-inf,inf,1.0


In [282]:

# Assuming bp.records is already a DataFrame
filtered_df = bp.records[bp.records['stocks'] == 'Stock_23']

# Display the filtered DataFrame
print(filtered_df)

       stocks time    level  marginal    lower  upper  scale
286  Stock_23    0      0.0       NaN  20000.0    inf    1.0
287  Stock_23    1  20000.0       NaN  20000.0    inf    1.0
288  Stock_23    2      0.0       NaN  20000.0    inf    1.0
289  Stock_23    3      0.0       NaN  20000.0    inf    1.0
290  Stock_23    4      0.0       NaN  20000.0    inf    1.0
291  Stock_23    5      0.0       NaN  20000.0    inf    1.0
292  Stock_23    6      0.0       NaN  20000.0    inf    1.0
293  Stock_23    7      0.0       NaN  20000.0    inf    1.0
294  Stock_23    8      0.0       NaN  20000.0    inf    1.0
295  Stock_23    9      0.0       NaN  20000.0    inf    1.0
296  Stock_23   10      0.0       NaN  20000.0    inf    1.0
297  Stock_23   11      0.0       NaN  20000.0    inf    1.0
298  Stock_23   12      0.0       NaN  20000.0    inf    1.0
